In [121]:
import numpy as np
from matplotlib import pyplot as plt

def loaddata ():
    dataSet = [[0, 0,0,0,0,0, 'yes'],
               [1, 0,1,0,0,0,'yes'],
               [1, 0,0,0,0,0,'yes'],
               [0, 0,1,0,0,0,'yes'],
               [2, 0,0,0,0,0,'yes'],
               [0, 1,0,0,1,1,'yes'],
               [1, 1,0,1,1,1,'yes'],
               [1, 1,0,0,1,0, 'yes'],
               [1, 1,1,1,1,0,'no'],
               [0, 2,2,0,2,1,'no'],
               [2, 2,2,2,2,0,'no'],
               [2, 0,0,2,2,1,'no'],
               [0, 1,0,1,0,0, 'no'],
               [2, 1,1,1,0,0,'no'],
               [1, 1,0,0,1,1,'no'],
               [2, 0,0,2,2,0,'no'],
               [0, 0,1,1,1,0,'no']]
    feature_name = ['a1','a2','a3','a4','a5','a6']
    return dataSet, feature_name

In [122]:
##计算经验熵
def compute_empirical_entropy(dataSet):
    numEntries = len(dataSet)
    labelCount = {}
    ##经验熵
    empiricalEnt = 0.0
    
    #对当前每一个数据进行统计
    for entry in dataSet:
        currentLabel = entry[-1]
        if currentLabel not in labelCount.keys():
            labelCount[currentLabel] = 0

        labelCount[currentLabel] +=1

        
    for key in labelCount.keys():
        prob = labelCount[key]/numEntries
        empiricalEnt -= prob*np.log2(prob)
    return empiricalEnt;


In [123]:
#按照某一个特征的不同取值切分数据集
#dataSet 为数据集
#index为表示它是数据集中第index+1个特征（下标从0开始）
#value表示特征值
def split_dataSet(dataSet,index,value):
    retDataSet = []
    for entry in dataSet:
        if entry[index] == value:
            #如果当前数据的特征为value，将它这一特征项去掉，然后加到retDataSet中
            tmpEntry = entry[:index]
            tmpEntry.extend(entry[index+1:])
            retDataSet.append(tmpEntry)
    return retDataSet

In [124]:
#计算条件熵
def compute_conditional_entropy(dataSet,index):
    featureValues = [exmaple[index] for exmaple in dataSet]
    featureValues = set(featureValues)
    condEnt = 0.0
    
    for featureValue in featureValues:
        subdataSet = split_dataSet(dataSet,index,featureValue)
        prob = len(subdataSet)/len(dataSet)
        condEnt += prob*compute_empirical_entropy(subdataSet)
    
    return condEnt

    #计算某个特征的增益信息
def compute_infoGain(dataSet,index):
    return compute_empirical_entropy(dataSet) - compute_conditional_entropy(dataSet,index)


In [145]:
#选择增益信息最大的特征,并且返回它的下标
def chooseBestFeature(dataSet):
    
    #特征的数量
    featureNum = len(dataSet[0]) - 1
    #最优特征的下标
    bestFeatureIndex = -1
    #当前最大信息增益
    maxInfoGain = 0
    
    for i in range(featureNum):
        infoGain = compute_infoGain(dataSet,i)
        
        if(infoGain > maxInfoGain):
            maxInfoGain = infoGain
            bestFeatureIndex = i
    return bestFeatureIndex

In [146]:
##选择主要类
def majorityCnt(classList):
    classCount={}
    #统计classList中每个元素出现的次数
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote]=0
            classCount[vote]+=1
        #根据字典的值降序排列
        sortedClassCount=sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
        return sortedClassCount[0][0]


In [149]:
import copy
#构建决策树
def createTree(dataSet,labels):
    classList = [exmaple[-1] for exmaple in dataSet]
    
    #如果当前子集中所有数据的类型都一样，则递归停止，返回这个类
    if(classList.count(classList[0]) == len(dataSet)):
        return classList[0]
    
    #如果当前子集中只剩下一个特征，则停止递归，返回子集中数据最多的类别
    if(len(dataSet) == 1):
        return majorityCnt(classList)
    
    bestFeatureIndex = chooseBestFeature(dataSet)
    bestFeature = labels[bestFeatureIndex]
    
    ## 为了使下一层对labels的改变不影响上一层，我们对labels进行深拷贝得到labels，然后将labels2传到下一层
    labels2 = copy.deepcopy(labels)
    del(labels2[bestFeatureIndex])
    
    myTree = {bestFeature:{}}
    featureValues = set([exmaple[bestFeatureIndex] for exmaple in dataSet])
    
    for featureValue in featureValues:
        myTree[bestFeature][featureValue] = createTree(split_dataSet(dataSet,bestFeatureIndex,featureValue),labels2)
    return myTree

In [150]:
dataSet,labels = loaddata()
print(createTree(dataSet,labels))

{'a4': {0: {'a2': {0: 'yes', 1: {'a1': {0: 'yes', 1: {'a6': {0: 'yes', 1: 'no'}}}}, 2: 'no'}}, 1: {'a6': {0: 'no', 1: 'yes'}}, 2: 'no'}}
